## Migrating from Pyomo

Similar to the implementation in Pyomo, expressions and constraints can be created using a combination of a function and a set of coordinates to iterate over. For creating expressions, the function itself has to return a `ScalarLinearExpression` which can be obtained by selecting single values of the variables are combining them: 

In [ ]:
import pandas as pd

import linopy

m = linopy.Model()
coords = pd.RangeIndex(10), ["a", "b"]
x = m.add_variables(0, 100, coords, name="x")
x

In [ ]:
x.at[0, "a"]

.. important::
    The creation of scalar variables has changed in version `0.3.10` to use the `.at[]` method. When creating a `ScalarVariable` with the `[]` operator, a future warning is raised. The `[]` operator will reserver for integer and boolean indexing only, aligning to the xarray functionality. 



Such a `ScalarVariable` is very light-weight and can be used in functions in order to create expressions, just like you know it from `Pyomo`. The following function shows how:

In [ ]:
def bound(m, i, j):
    if i % 2:
        return (i / 2) * x.at[i, j]
    else:
        return i * x.at[i, j]


expr = m.linexpr(bound, coords)
expr

Note that the function's first argument has to be the model itself, even though it might not be used in the function.

This functionality is also supported by the `.add_constraints` function. When passing a function as a first argument, `.add_constraints` expects `coords` to by non-empty. The function itself has to return a `AnonymousScalarConstraint`, as done by 

In [ ]:
x.at[0, "a"] <= 3

In [ ]:
def bound(m, i, j):
    if i % 2:
        return (i / 2) * x.at[i, j] >= i
    else:
        return i * x.at[i, j] == 0.0


con = m.add_constraints(bound, coords=coords)
con